In [10]:
#Load the eye data
import pandas as pd
import matplotlib.pyplot as plt

eye_data = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/sub-03/eyetracking/sub-03_task-DeepMReyeClosed_run_01_eyedata.tsv.gz", compression='gzip', delimiter='\t')
eye_data = eye_data[['timestamps','x', 'y']].to_numpy()

In [2]:
import pandas as pd
import numpy as np
def adapt_evaluation(participant_evaluation):
    pred_y = participant_evaluation["pred_y"]
    pred_y_median = np.nanmedian(pred_y, axis=1)
    pred_uncertainty = abs(participant_evaluation["euc_pred"])
    pred_uncertainty_median = np.nanmedian(pred_uncertainty, axis=1)
    df_pred_median = pd.DataFrame(
        np.concatenate(
            (pred_y_median, pred_uncertainty_median[..., np.newaxis]), axis=1),
        columns=["X", "Y", "Uncertainty"],
    )
    # With subTR
    subtr_values = np.concatenate((pred_y, pred_uncertainty[..., np.newaxis]),
                                  axis=2)
    index = pd.MultiIndex.from_product(
        [range(subtr_values.shape[0]),
         range(subtr_values.shape[1])],
        names=["TR", "subTR"])
    df_pred_subtr = pd.DataFrame(subtr_values.reshape(-1,
                                                      subtr_values.shape[-1]),
                                 index=index,
                                 columns=["X", "Y", "pred_error"])

    return df_pred_median, df_pred_subtr

In [3]:
evaluation_closed = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed.npy", allow_pickle=True).item()
sub_03_closed = evaluation_closed["/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data/sub-03_DeepMReyeClosed_label.npz"]
df_pred_median_3, df_pred_subtr_3 = adapt_evaluation(sub_03_closed)
sub_03_run_01_closed_X = np.array(df_pred_subtr_3['X'][:3130])
sub_03_run_01_closed_Y = np.array(df_pred_subtr_3['Y'][:3130])

In [4]:
import numpy as np
from scipy.signal import resample

def downsample_to_targetrate(original_data, eyetracking_rate, target_rate):
    """
    Downsample eye-tracking data to a specified target rate.

    Args:
        original_data (np.array): Eye-tracking data array with columns for timestamp, X, Y, and pupil size.
        eyetracking_rate (int): Sampling rate of the original data.
        target_rate (float): Desired target sampling rate.

    Returns:
        np.array: Downsampled eye-tracking data.
    """
    # Remove NaNs
    def remove_nans(data):
        for col in range(data.shape[1]):
            nan_indices = np.isnan(data[:, col])
            if np.any(nan_indices):
                data[nan_indices, col] = np.interp(
                    np.flatnonzero(nan_indices),
                    np.flatnonzero(~nan_indices),
                    data[~nan_indices, col]
                )
        return data

    original_data = remove_nans(original_data)

    # Calculate total number of data points in target rate
    eyetracking_in_sec = len(original_data) / eyetracking_rate
    total_target_points = int(eyetracking_in_sec * target_rate)

    # Resample data columns
    downsampled_t = np.linspace(
        original_data[0, 0], original_data[-1, 0], total_target_points
    )
    downsampled_x = resample(original_data[:, 1], total_target_points)
    downsampled_y = resample(original_data[:, 2], total_target_points)

    # Stack into final array
    downsampled_data = np.column_stack((downsampled_t, downsampled_x, downsampled_y))

    return downsampled_data



In [11]:
eye_data_downsampled = downsample_to_targetrate(eye_data, 1000, 8.33)

In [12]:
from scipy.signal import detrend, resample
import pandas as pd
import numpy as np
design_matrix = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/exp_design/sub-03/sub-03_ses-02_task-DeepMReyeClosed_run-01_design_matrix.tsv", sep ="\t")
task_1_trials = np.array(design_matrix["task_eyes_open_task"])
task_3_trials = np.array(design_matrix["task_no_stimulus_task"])
task_4_trials = np.array(design_matrix["task_eyes_closed_task"])

resampled_task_1_trials = resample(task_1_trials, len(eye_data_downsampled[:,1]))
resampled_task_3_trials = resample(task_3_trials, len(eye_data_downsampled[:,1]))
resampled_task_4_trials = resample(task_4_trials, len(eye_data_downsampled[:,1]))

task_1_trials_bool = resampled_task_1_trials > 0.5
task_3_trials_bool = resampled_task_3_trials > 0.5
task_4_trials_bool = resampled_task_4_trials > 0.5

eye_data_task_1 = eye_data_downsampled[task_1_trials_bool]
eye_data_task_3 = eye_data_downsampled[task_3_trials_bool]
eye_data_task_4 = eye_data_downsampled[task_4_trials_bool]


In [15]:
sub_03_run_01_closed_X_task_1 = sub_03_run_01_closed_X[:int(len(eye_data_downsampled))][task_1_trials_bool]
sub_03_run_01_closed_Y_task_1 = sub_03_run_01_closed_Y[:int(len(eye_data_downsampled))][task_1_trials_bool]

In [23]:
sub_03_run_01_closed_X_task_4 = sub_03_run_01_closed_X[:int(len(eye_data_downsampled))][task_4_trials_bool]
sub_03_run_01_closed_Y_task_4 = sub_03_run_01_closed_Y[:int(len(eye_data_downsampled))][task_4_trials_bool]

In [17]:
print(sub_03_run_01_closed_X_task_1.shape)
print(eye_data_task_1.shape)

(743,)
(743, 3)


In [25]:
import plotly.graph_objects as go
import numpy as np
import os

N = len(eye_data_task_1)
x = sub_03_run_01_closed_X_task_4
#[::150]
y = sub_03_run_01_closed_Y_task_4

# Create output directory for frames
output_dir = f"/Users/sinakling/Desktop/frames_task_4"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Capture each frame and save as PNG
for k in range(N): 
    fig = go.Figure( 
        data=[ go.Scatter(x=x[:k+1], 
                            y=y[:k+1], 
                            mode="lines", 
                            line=dict(width=2, color="black")), ], 
                            layout=go.Layout( xaxis=dict(range=[-15, 15], 
                                                        autorange=False, 
                                                        zeroline=False), 
                                                        yaxis=dict(range=[-15, 15], 
                                                                    autorange=False, 
                                                                    zeroline=False), 
                                                                    title_text="DeepMReyeClosed Task 4 FT", 
                                                                    hovermode="closest", 
                                                                    template='simple_white' ), ) 
    # Adjust layout for a square plot 
    fig.update_layout( autosize=False, width=800, height=800, margin=dict( l=50, r=50, b=100, t=100 ) ) 

    # Save each frame as a PNG file 
    fig.write_image(f"{output_dir}/frame_{k:03d}.png")

# Display the initial figure for verification
fig.show()
